In [ ]:
import io
import socket
import struct  
import logging
import socketserver
import threading 
from http import server
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 8000))  
server_socket.listen(0)  

class StreamingOutput(object):
    def __init__(self):
        self.frame = None
        self.buffer = io.BytesIO()
        self.condition = threading.Condition()
        self.receive_thread = threading.Thread(target=self.receiver)
        self.receive_thread.daemon = True
        self.receive_thread.start() 
        
    def receiver(self): 
        while True:
            print('Waiting for RPi streaming.')
            try: 
                connection = server_socket.accept()[0].makefile('rb')
                while True:
                    image_len = struct.unpack('<L', connection.read(struct.calcsize('<L')))[0]
                    if not image_len:
                        break 
                    img_IO = io.BytesIO()
                    img_IO.write(connection.read(image_len)) 
                    img_IO.seek(0) 
                    with self.condition:
                        self.frame = img_IO.read()
                        self.condition.notify_all()   
            finally:
                print('Streaming interrupted, restart connection.')
                connection.close()

class StreamingHandler(server.BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/stream.mjpg')
            self.end_headers() 
        elif self.path == '/stream.mjpg':
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()
            try:
                while True:
                    with output.condition:
                        output.condition.wait()
                        frame = output.frame
                    self.wfile.write(b'--FRAME\r\n')
                    self.send_header('Content-Type', 'image/jpeg')
                    self.send_header('Content-Length', len(frame))
                    self.end_headers()
                    self.wfile.write(frame)
                    self.wfile.write(b'\r\n')
            except Exception as e:
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()

class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

def streaming():
    output = StreamingOutput()  
    server = StreamingServer(('', 8001), StreamingHandler)
    server.serve_forever()  

t = threading.Thread(target=streaming)
t.daemon = True
t.start() 

Waiting for RPi streaming.


192.168.0.123 - - [08/Sep/2019 08:31:55] "GET /stream.mjpg HTTP/1.1" 200 -


Streaming interrupted, restart connection.
Waiting for RPi streaming.


192.168.0.123 - - [08/Sep/2019 08:33:09] "GET /stream.mjpg HTTP/1.1" 200 -


Streaming interrupted, restart connection.
Waiting for RPi streaming.
